## Regression Trees

### Boosting

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from  sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.metrics import mean_squared_error as MSE

In [ ]:
data = pd.read_csv('../data/Data-train.csv',index_col=0)
data

,productID,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,weekday,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,sales
0,68,28,3,100,9,1.561599,799,237.206500,186.956967,11.785714,4,35.498327,0.114401,3.046749,0.562923,0.206291,2.173474,3.628150,2.181482,14
1,31,6,3,100,2,7.747685,352,178.540538,129.123143,25.785714,7,25.633478,0.000000,3.005766,0.000000,0.444095,1.927769,3.058834,1.511026,2
2,27,19,3,60,2,3.899665,293,119.066221,59.136199,21.357143,6,32.228120,0.158911,2.935705,0.293093,0.132959,2.324772,2.650857,1.665037,16
3,52,4,3,80,1,1.585429,537,139.452545,68.572100,7.285714,5,29.304317,0.186522,2.810752,0.242181,0.000000,2.192938,2.026259,2.481082,13
4,3,28,3,100,14,4.623051,6668,298.106532,244.047761,296.214286,5,29.132081,0.300183,3.032527,0.391279,0.103062,2.135294,2.736957,1.979200,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571,4,29,4,100,3,1.610909,1558,89.371417,41.336335,34.142857,6,38.176047,0.434893,2.805286,0.652032,0.678596,2.262897,3.318826,2.247318,131
1572,24,26,3,40,3,13.657958,74,80.494579,59.261375,5.214286,4,20.134348,0.019198,2.969021,0.052638,0.065188,2.072099,1.039334,5.042053,1
1573,24,26,3,40,2,5.449706,353,76.912038,48.459064,5.000000,1,35.180962,0.260036,2.425446,0.701358,0.190540,2.465418,3.155204,2.745468,11
1574,46,29,2,50,7,9.496754,1740,69.920092,50.999694,59.857143,6,31.277144,0.639961,3.125656,0.678429,0.319564,2.269736,2.769317,2.308552,35


Log transformation on y response. Set train_size=80%.

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.001, 0.002, 0.005, 0.01, 0.02, 0.05]
MSE_test=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=10, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[6258.773199718557,
 4812.396460384886,
 1803.4618221056085,
 1023.7138272484258,
 971.1373725914152,
 937.0686581354335]

Tuning learning rate, conduct 5-fold cross validation.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

X = data.drop(["sales"], axis=1)
y = data.sales
y_log = np.log(y)
rate = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]
MSE_test = []


kf = KFold(n_splits=5, random_state=24, shuffle=True)

for i in rate:
    temp_mse = []
    for train_index, test_index in kf.split(X):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_log.iloc[train_index], y_log.iloc[test_index]

        gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,random_state=24)
        gbc.fit(X_train, y_train)

        y_pred = np.exp(gbc.predict(X_test))
        temp_mse.append(mean_squared_error(y_pred, np.exp(y_test)))

    MSE_test.append(np.mean(temp_mse))

print(MSE_test)

[6160.751197974872, 4818.856357224567, 2171.696650549512, 1165.197831487868, 876.7036248753699, 813.7326349638427]


Train MSE at learning rate=0.05

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

MSE_train=[]
gbc = GradientBoostingRegressor()
gbc = GradientBoostingRegressor(learning_rate=0.05, n_estimators=500, random_state=24)
gbc.fit(X_train,y_train_log)
y_score=np.exp(gbc.predict(X_train))
MSE_train.append(mean_squared_error(y_score,y_train))
MSE_train

[204.26871129814157]

In [ ]:
y_pred=np.exp(gbc.predict(X_test))
mean_squared_error(y_pred,y_test)

904.9389093229586

This may be overfitting because of small training MSE and large test MSE.

Tuning max_depth = [3,4,5,6,7,8,9,10]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

X = data.drop(["sales"], axis=1)
y = data.sales
y_log = np.log(y)
depth = [3,4,5,6,7,8,9,10]
MSE_test = []


kf = KFold(n_splits=5, random_state=24, shuffle=True)

for i in depth:
    temp_mse = []
    for train_index, test_index in kf.split(X):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y_log.iloc[train_index], y_log.iloc[test_index]

        gbc = GradientBoostingRegressor(learning_rate=0.05, n_estimators=500,max_depth=i,random_state=24)
        gbc.fit(X_train, y_train)

        y_pred = np.exp(gbc.predict(X_test))
        temp_mse.append(mean_squared_error(y_pred, np.exp(y_test)))

    MSE_test.append(np.mean(temp_mse))

print(MSE_test)

[813.7326349638427, 809.7216099453765, 930.6867095669984, 1007.7817154157626, 1171.6541498113456, 1300.0876294474297, 1404.1306454814353, 1499.6418671854167]


The best max_depth =3. Tune learning_rate again

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.05,0.06,0.07,0.08,0.09,0.1]
MSE_test=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[904.9389093229586,
 929.682425914903,
 884.5191297538252,
 886.9716151724971,
 838.4233214755786,
 774.6576419329725]

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.05,0.06,0.07,0.08,0.09,0.1]
MSE_train=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_score=np.exp(gbc.predict(X_train))
    MSE_train.append(MSE(y_score,y_train))
MSE_train

[204.26871129814157,
 163.68620267182774,
 134.04149827685214,
 101.25393822000204,
 83.61837444646311,
 76.32289107893972]

Training and test MSE decreases as learning rate increases.

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.1,0.2,0.3,0.4,0.5]
MSE_test=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[774.6576419329725,
 886.9939357168398,
 576.1322639590942,
 936.5516608862806,
 952.6654695964947]

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.1,0.2,0.3,0.4,0.5]
MSE_train=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_train))
    MSE_train.append(mean_squared_error(y_predict,y_train))
MSE_train

[76.32289107893972,
 17.005193145884682,
 3.3679693864411178,
 0.7097761908633977,
 0.20019187747527648]

Results above also shows overfitting.

Best learning_rate=0.3

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)


from  sklearn.ensemble import GradientBoostingRegressor
rate=[0.297,0.298,0.299,0.3,0.301,0.302,0.303,0.304]
MSE_test=[]
score=[]
for i in rate:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=i, n_estimators=500,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[978.9888515594915,
 803.8384273360767,
 831.9677039775238,
 576.1322639590942,
 969.2910253919865,
 1062.9562021802476,
 976.0553211721156,
 622.8453198840516]

Also best learning rate=0.3

Tuning n_estimator:

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from  sklearn.ensemble import GradientBoostingRegressor
estimator=[500,600,700,800,900,1000]
MSE_test=[]
score=[]
for i in estimator:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=i,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[576.1322639590942,
 588.4787471384526,
 589.9679337440243,
 591.2526527460147,
 593.6791788434609,
 596.7906479424175]

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from  sklearn.ensemble import GradientBoostingRegressor
estimator=[100,200,300,400]
MSE_test=[]
score=[]
for i in estimator:
    gbc = GradientBoostingRegressor()
    gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=i,max_depth=3, random_state=24)
    gbc.fit(X_train,y_train_log)
    y_predict=np.exp(gbc.predict(X_test))
    MSE_test.append(mean_squared_error(y_predict,y_test))
MSE_test

[702.5363093784009, 561.2453646080444, 563.2173138444286, 570.127932036661]

The best n_estimators=200.

Final model:

In [ ]:
gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=200,max_depth=3, random_state=24)
gbc.fit(X_train,y_train_log)

Importance Ranking

In [ ]:
importance = gbc.feature_importances_
indices = np.argsort(importance)[::-1]
features=X.columns
for f in range(X.shape[1]):
    print(("%2d) %-*s %f" % (f + 1, 30, features[indices[f]], importance[indices[f]])))

 1) clickVolume                    0.881542
 2) avgFinalUnitPrice              0.044821
 3) gender                         0.013868
 4) avgOriginalUnitPrice           0.013127
 5) meanEducation                  0.009808
 6) maritalStatus                  0.008893
 7) ma14SalesVolume                0.007253
 8) plus                           0.007170
 9) meanPurchasePower              0.004585
10) attribute4                     0.002259
11) meanAge                        0.001796
12) productID                      0.001694
13) weekday                        0.001349
14) meanCityLevel                  0.001187
15) meanUserLevel                  0.000208
16) attribute3                     0.000181
17) brandID                        0.000122
18) attribute2                     0.000097
19) attribute1                     0.000038


In [ ]:
testdata = pd.read_csv('../data/Data-test.csv',index_col=0)
gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=200,max_depth=3, random_state=24)
gbc.fit(X_train,y_train_log)
predictions = gbc.predict(testdata)
predictions = np.exp(predictions)
testdata['Sales'] = predictions
testdata
testdata.to_csv("..\data\testdata.csv", index=False)

5-fold cross validation mean MSE:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

X = data.drop(["sales"], axis=1)
y = data.sales
kf = KFold(n_splits=5, random_state=24, shuffle=True)

MSE_test = []

for train_index, test_index in kf.split(X):
    # Split the data to training set and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Take the logarithm of the training part of y in each fold
    y_train_log = np.log(y_train)

    # Fit model to data
    gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=200,max_depth=3,random_state=24)
    gbc.fit(X_train, y_train_log)

    # Use the model to predict and compute the MSE
    y_pred = np.exp(gbc.predict(X_test))
    mse = mean_squared_error(y_test, y_pred)
    MSE_test.append(mse)
    print(f"MSE for this fold: {mse}")

# Print the average MSE across all folds
print(f"Average MSE: {np.mean(MSE_test)}")

MSE for this fold: 711.296990477735
MSE for this fold: 1021.7968468703847
MSE for this fold: 447.9106409316581
MSE for this fold: 1157.50603058594
MSE for this fold: 993.2725976816943
Average MSE: 866.3566213094824


### Random Forest

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor()

alpha= [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
MSE_test_2=[]
score=[]
for i in alpha:
    RF = RandomForestRegressor(n_estimators=100, random_state=24, ccp_alpha=i)
    RF.fit(X_train,y_train_log)
    y_predict=np.exp(RF.predict(X_test))
    MSE_test_2.append(MSE(y_predict,y_test))
MSE_test_2

[1926.975024585017,
 2672.889075009585,
 3259.0842717124588,
 3403.309339687088,
 3444.2343974063106,
 3482.749197838026,
 3483.499771928011]

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor()

alpha= [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
MSE_train_2=[]
score=[]
for i in alpha:
    RF = RandomForestRegressor(n_estimators=100, random_state=24, ccp_alpha=i)
    RF.fit(X_train,y_train_log)
    y_predict=np.exp(RF.predict(X_train))
    MSE_train_2.append(MSE(y_predict,y_train))
MSE_train_2

[1514.6980811827812,
 2097.6631639149277,
 2629.0145954326463,
 2769.166656189364,
 2811.5351191877926,
 2851.55381715856,
 2851.989292268]

Tuning ccp_alpha:

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from sklearn.ensemble import RandomForestRegressor

alpha= [0.005,0.006,0.007,0.008,0.009,0.01]
MSE_test_2=[]
for i in alpha:
    RF = RandomForestRegressor(n_estimators=100, random_state=24, ccp_alpha=i)
    RF.fit(X_train,y_train_log)
    y_predict=np.exp(RF.predict(X_test))
    MSE_test_2.append(MSE(y_predict,y_test))
MSE_test_2

[1724.6442802365802,
 1803.1220076033183,
 1852.4615793172388,
 1878.6062771118964,
 1905.402438919809,
 1926.975024585017]

Set ccp_alpha=0.005 and tune max_depth.

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from sklearn.ensemble import RandomForestRegressor
max_depth=[3,4,5,6,7,8,9,10]
MSE_test_2=[]
for i in depth:
    RF = RandomForestRegressor(n_estimators=100, random_state=24, max_depth=i,ccp_alpha=0.005)
    RF.fit(X_train,y_train_log)
    y_predict=np.exp(RF.predict(X_test))
    MSE_test_2.append(MSE(y_predict,y_test))
MSE_test_2

[2013.5610009976103,
 1744.9394764308165,
 1729.6703811635646,
 1725.2489624766956,
 1724.546334514231,
 1724.6282164163529,
 1724.6888922104713,
 1724.6033010293975]

The best max_depth=5.

Tuning n_estimators:

In [ ]:
X = data.drop(["sales"], axis=1)
y = data.sales
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=25)
y_train_log= np.log(y_train)

from sklearn.ensemble import RandomForestRegressor
estimators=[100,200,300,400,500,700,800,900,1000]
MSE_test_2=[]
for i in estimators:
    RF = RandomForestRegressor(n_estimators=i, random_state=24, max_depth=5,ccp_alpha=0.005)
    RF.fit(X_train,y_train_log)
    y_predict=np.exp(RF.predict(X_test))
    MSE_test_2.append(MSE(y_predict,y_test))
MSE_test_2

[1729.6703811635646,
 1720.9231973919793,
 1716.4051732655123,
 1693.9739622011853,
 1705.1807259632483,
 1699.254256775518,
 1696.8471076177836,
 1690.9566346365723,
 1695.2270182654888]

The best n_estimators=900

Final random forest model:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X = data.drop(["sales"], axis=1)
y = data.sales
kf = KFold(n_splits=5, random_state=24, shuffle=True)

MSE_test = []

for train_index, test_index in kf.split(X):
    # Split the data to training set and testing set
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Take the logarithm of the training part of y in each fold
    y_train_log = np.log(y_train)

    # Fit model to data
    RF = RandomForestRegressor(n_estimators=900, random_state=24, max_depth=5,ccp_alpha=0.005)
    RF.fit(X_train, y_train_log)

    # Use the model to predict and compute the MSE
    y_pred = np.exp(RF.predict(X_test))
    mse = mean_squared_error(y_test, y_pred)
    MSE_test.append(mse)
    print(f"MSE for this fold: {mse}")

# Print the average MSE across all folds
print(f"Average MSE: {np.mean(MSE_test)}")

MSE for this fold: 1466.2208574153299
MSE for this fold: 1131.7837144998884
MSE for this fold: 1181.339591091809
MSE for this fold: 2066.957465128353
MSE for this fold: 1755.5150209443996
Average MSE: 1520.363329815956


### RSS for Boosting

In [ ]:
traindata = pd.read_csv('..\data\Data-train.csv',index_col=0)
X = data.drop(["sales"], axis=1)
y = data.sales
y_log= np.log(y)
gbc = GradientBoostingRegressor(learning_rate=0.3, n_estimators=200,max_depth=3,random_state=24)
gbc.fit(X, y_log)

# Use the model to predict and compute the MSE
y_pred = np.exp(gbc.predict(X))
residuals = y - y_pred

# 计算残差平方和(RSS)
RSS = np.sum(residuals**2)
print('残差平方和(RSS):', RSS)


残差平方和(RSS): 133116.0222414814
